# <span style="color:#3E6BCE">1) 주제</span>

Web crawling과 random forest를 활용한 축구 경기 결과 예측 모델 제작

# <span style="color:#3E6BCE">2) 프로젝트 진행사항</span>

**Web crawling**
>  Web crawling에서 자주 사용되는 파이썬 라이브러리인 selenium과 pandas를 활용해 crawling을 진행하였습니다. 동적인 웹페이지에서 crawling을 진행하고자 하여 beautifulsoup4가 아닌 selenium을 활용하였고, 이를 통해 추출한 대용량 데이터를 쉽게 다루기 위해 pandas를 이용해 프로젝트를 진행하였습니다. 그 결과, 현재까지 webdriver로 url을 호출해, 필요한 데이터들을 추출하여 csv파일을 생성하였습니다.<br><br>
또한 데이터 추출은 잉글랜드 프리미어리그 공식 사이트(<https://www.premierleague.com/home>)를 통해 진행하였습니다. 해당 코드와 csv파일은 아래와 같습니다.

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
from datetime import datetime
import pandas as pd

errors = []
season = []

#19-20시즌 경기 고유 번호 46605~46985
for id in range(46605, 46607):
    url = f'https://www.premierleague.com/match/{id}'
    option = Options()
    option.headless = False
    driver = webdriver.Chrome(options=option)
    driver.get(url)
    sleep(3)
    
    # 데이터 가져오기
    try:
        #날짜 데이터
        date = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((
            By.XPATH, '//*[@id="mainContent"]/div/section[2]/div[2]/section/div[1]/div/div[1]/div[1]'))).text
        date = datetime.strptime(date, '%a %d %b %Y').strftime('%m/%d/%Y')
        
        #해당 경기 팀과 결과 데이터
        home_team = driver.find_element_by_xpath(
            '//*[@id="mainContent"]/div/section[2]/div[2]/section/div[3]/div/div/div[1]/div[1]/a[2]/span[1]').text
        away_team = driver.find_element_by_xpath(
            '//*[@id="mainContent"]/div/section[2]/div[2]/section/div[3]/div/div/div[1]/div[3]/a[2]/span[1]').text
        scores = driver.find_element_by_xpath(
            '//*[@id="mainContent"]/div/section[2]/div[2]/section/div[3]/div/div/div[1]/div[2]/div/div').text
        home_score = scores.split('-')[0]
        away_score = scores.split('-')[1]
        
        #stats 데이터 부분 클릭
        elem = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='mainContent']/div/section[2]/div[2]/div[2]/div[1]/div/div/ul/li[3]")))
        elem.click()
        sleep(3)
        
        #html에 있는 table값 가져오기
        dfs = pd.read_html(driver.page_source)
        stats = dfs[-1]

        driver.quit()
    
    #예외 처리
    except:
        driver.quit()
        errors.append(id)
        continue

    # stats 데이터 처리하기
    home_stats = {}
    away_stats = {}

    home_series = stats[home_team]
    away_series = stats[away_team]
    stats_series = stats['Unnamed: 1']

    for row in zip(home_series, stats_series, away_series):
        stat = row[1].replace(' ', '_').lower()
        home_stats[stat] = row[0]
        away_stats[stat] = row[2]

    stats_check = ['possession_%', 'shots_on_target', 'shots', 'touches', 'passes','tackles', 'clearances', 'corners', 'offsides', 'yellow_cards','red_cards', 'fouls_conceded']

    for stat in stats_check:
        if stat not in home_stats.keys():
            home_stats[stat] = 0
            away_stats[stat] = 0

    # 전체 데이터 저장하기
    match = [date, home_team, away_team, home_score, away_score, home_stats['possession_%'], away_stats['possession_%'],
             home_stats['shots_on_target'], away_stats['shots_on_target'], home_stats['shots'], away_stats['shots'],
             home_stats['touches'], away_stats['touches'], home_stats['passes'], away_stats['passes'],
             home_stats['tackles'], away_stats['tackles'], home_stats['clearances'], away_stats['clearances'],
             home_stats['corners'], away_stats['corners'], home_stats['offsides'], away_stats['offsides'],
             home_stats['yellow_cards'], away_stats['yellow_cards'], home_stats['red_cards'], away_stats['red_cards'],
             home_stats['fouls_conceded'], away_stats['fouls_conceded']]

    season.append(match)

    # 데이터 추출하기
    columns = ['date', 'home_team', 'away_team', 'home_score', 'away_score']

    for stat in stats_check:
        columns.append(f'home_{stat}')
        columns.append(f'away_{stat}')

    dataset = pd.DataFrame(season, columns=columns)
    dataset.to_csv('C:/Users/ballj/OneDrive/바탕 화면/Premier_league_19_20.csv', index=False)

    dataset

![csv파일 캡처본](https://raw.githubusercontent.com/jeongmin1217/kakao-clone-2021/gh-pages/%EC%BA%A1%EC%B2%98.PNG)

# <span style="color:#3E6BCE">3) 프로젝트 계획</span>

- **Random forest 기법 활용 및 경기 결과 도출**
> 위의 csv 파일을 바탕으로, 각 팀의 득점을 target attribute로 설정해 득점과 유의미한 관계를 지녔던 데이터를 random forest 기법을 통해 도출해 낼 예정입니다. 그리고 이를 통해, 득점 수를 예측해 최종적으로 경기 결과를 예측해내는 프로젝트를 진행할 계획입니다.<br><br>
또한 해당 csv파일은 작년 시즌인 2019-2020 시즌 데이터로, 이를 통해 올해 2020-2021 시즌에 있었던 경기 결과를 예측하는 실험을 해, 실제 경기 결과와 비교하여 예측률을 측정할 예정입니다. 그리고 이를 바탕으로 순위표를 작성해, 실제 순위표와 어느 정도 비슷한지 비교해 볼 예정입니다. 추후 진행할 이 과정을 위해 이번 시즌 실제 순위표를 crawling 해 보았습니다. 코드는 아래와 같습니다.

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests

driver = webdriver.Chrome()
driver.implicitly_wait(3)

naver_wfootball = "https://sports.news.naver.com/wfootball/record/index.nhn?category=epl&year=2021"
driver.get(naver_wfootball)

page = driver.page_source
premi_team_rank_list =  BeautifulSoup(page,"html.parser")
team_rank_list = premi_team_rank_list.select('#wfootballTeamRecordBody>table>tbody>tr')

for team in team_rank_list:
    num = team.select('.num > div.inner > strong')[0].text
    name = team.select('.name')[0].text
    print(num +"위: "+name)

1위: 맨체스터 시티 FC
2위: 맨체스터 유나이티드 FC
3위: 리버풀 FC
4위: 첼시 FC
5위: 레스터 시티 FC
6위: 웨스트햄 유나이티드 FC
7위: 토트넘 홋스퍼 FC
8위: 아스널 FC
9위: 리즈 유나이티드 FC
10위: 에버턴 FC
11위: 아스톤 빌라 FC
12위: 뉴캐슬 유나이티드 FC
13위: 울버햄튼 원더러스 FC
14위: 크리스탈 팰리스 FC
15위: 사우샘프턴 FC
16위: 브라이튼 앤 호브 알비온 FC
17위: 번리 FC
18위: 풀럼 FC
19위: 웨스트 브로미치 앨비언 FC
20위: 셰필드 유나이티드 FC


# <span style="color:#3E6BCE">4) 프로젝트 보완점</span>

> 해당 프로젝트에서는 기존에 crawling을 진행하려 하였던 <https://www.whoscored.com/> 사이트에서 먼저 crawling을 시도하였습니다. 하지만 해당 사이트에서 데이터 parsing을 위해 beautifulsoup4를 사용하던 중, forbidden error가 발생하여 url에 접속할 수 없게 되었고, 이로 인해 다른 사이트에서 selenium을 활용해 데이터를 추출해오며, 처음 계획했던 데이터와는 조금 다른 데이터들을 통해 경기 결과 예측을 시도하게 되었습니다. 현재 추출에 성공한 데이터로도 유의미한 예측을 진행할 수 있지만, 더욱 예측률을 높이기 위해서는 기존의 사이트에서 weakness 데이터를 추출하는 것이 도움이 된다고 생각해, 최종 보고서 제출 전까지 발생했던 문제를 해결하여 보다 높은 예측률을 지닌 모델을 완성하기 위해 노력할 예정입니다.